# Modelling patch-clamp experiments: simplified models

In the previous notebooks we presented a model of patch-clamp experiments.
Here, we explore how it can be simplified for parameter estimation purposes.

We start from the main schematic and equations.

<img src="resources/patch-comp-6-full.png" style="margin:auto" />

_**Figure 1**: The full patch-clamp schematic_

\begin{align}
2.1. && C_m\dot{V}_m = \frac{V_p + E_\text{off}^\dagger - V_m}{R_s} - \frac{V_m - E_\text{leak}}{R_\text{leak}} - I
\end{align}

\begin{align}
2.2. && (C_p+C_f)\dot{V}_p = \frac{V_o - V_p}{R_f} - \frac{V_p + E_\text{off}^\dagger - V_m}{R_s} + C_f\dot{V}_o + C_m^* \dot{V}_\text{est} + C_p^* \dot{V}_\text{ref}
\end{align}

\begin{align}
2.3. && \tau_a \dot{V}_o = V_\text{ref} - V_p
\end{align}

\begin{align}
2.4. && \dot{V}_\text{est} &= \frac{V_c - V_\text{est}}{(1 - \beta)R_s^*C_m^*}   
\end{align}

\begin{align}
2.5. && \tau_\text{sum}\dot{V}_\text{ref} = V_c + \alpha R_s^* I_\text{obs} + \beta R_s^* C_m^* \dot{V}_\text{est} - V_\text{ref}
\end{align}

\begin{align}
2.6. && R_f I_\text{obs} = V_o - V_\text{ref}
\end{align}

## Op-amp speed

## Fast capacitance

## Conclusion